In [1]:
import sys
sys.path.append('../../../')

import pandas as pd

import keras

from utils.predictor import PredictorToolkit
from utils.dataset import TreadingDataset
from utils.strategy import TradeStrategy
from utils.backtester import Backtester
from utils.field_processing import create_ts_from_df
import warnings

warnings.filterwarnings('ignore')

2025-03-09 02:55:50.278441: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
SEQUENCE_LENGTH = 9
SHIFT_SIZE = 3

# Loading the model

In [3]:
model = keras.models.load_model('../../../models/test/best_model_EURUSD.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 512)            │     1,099,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,311,669 (12.63 MB)

 Trainable params: 1,103,889 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,207,780 (8.42 MB)

# Loading the previous sequences

In [4]:
base_df = pd.read_csv('../../../btc_last_test.csv', sep='\t', usecols=['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>'])
# base_df = pd.read_csv('../../../EURUSD_H1_202403_2025.csv', sep='\t', usecols=['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>'])
base_df.rename(columns={
    '<OPEN>': 'OPEN', 
    '<HIGH>': 'HIGH', 
    '<LOW>': 'LOW', 
    '<CLOSE>': 'CLOSE'
}, inplace=True)

base_df = create_ts_from_df(base_df, '<DATE>', '<TIME>', 'DATETIME')
base_df.tail()

,OPEN,HIGH,LOW,CLOSE
DATETIME,,,,
2025-03-07 19:00:00,1.08589,1.08644,1.08319,1.08434
2025-03-07 20:00:00,1.08432,1.08442,1.08256,1.08398
2025-03-07 21:00:00,1.08397,1.08440,1.08333,1.08425
2025-03-07 22:00:00,1.08424,1.08492,1.08399,1.08462
2025-03-07 23:00:00,1.08461,1.08506,1.08320,1.08325


In [5]:
df = base_df.loc[:'2025-02-03 08:00:00']
df.tail()

,OPEN,HIGH,LOW,CLOSE
DATETIME,,,,
2025-02-03 04:00:00,1.02378,1.02379,1.02210,1.02235
2025-02-03 05:00:00,1.02236,1.02247,1.02122,1.02214
2025-02-03 06:00:00,1.02213,1.02396,1.02205,1.02265
2025-02-03 07:00:00,1.02266,1.02430,1.02256,1.02401
2025-02-03 08:00:00,1.02401,1.02515,1.02394,1.02431


# Generate the next close price 

In [6]:
dataset = TreadingDataset(df[['CLOSE']])
dataset.prepare_data()

In [7]:
last_prev_lines = dataset.feature_sequence[-1]
tk = PredictorToolkit(dataset.target_scaler, model, SEQUENCE_LENGTH)

for _ in range(9):
    last_prev_lines = dataset.feature_sequence[-1]
    entry_price = dataset.df[['CLOSE']].iloc[-1:].values[0][0]
    new_df = tk.predict_the_next_price(dataset.df[['CLOSE']], entry_price, last_prev_lines)
    dataset.update_df(new_df)

dataset.calculate_atr(base_df, period=14)
tk.predict_df

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


,ENTRY_PRICE,PREDICTED_CLOSE
2025-02-03 09:00:00,1.024310,1.028349
2025-02-03 10:00:00,1.028349,1.028318
2025-02-03 11:00:00,1.028318,1.028293
2025-02-03 12:00:00,1.028293,1.028214
2025-02-03 13:00:00,1.028214,1.026950
2025-02-03 14:00:00,1.026950,1.026764
2025-02-03 15:00:00,1.026764,1.024791
2025-02-03 16:00:00,1.024791,1.023952
2025-02-03 17:00:00,1.023952,1.024141


# Make the trade strategy

## Stratégie avec un ratio perte/gain (Ratio Risk/Reward) de 1:2

perte de 10 pips pour gagner 20 pips
- Stop-loss pips = 0.0010
- Take-profit pips (Stop-loss pips * 2) = 0.0020


In [8]:
dataset.df.tail()

,CLOSE,EMA,RSI,MACD,SIGNAL_LINE,UPPER_BAND,LOWER_BAND,BOLLINGER_PERCENT_B,ATR
2025-02-03 13:00:00,1.026950,1.028982,29.517767,-0.002762,-0.003332,1.042369,1.015881,0.417917,0.003250
2025-02-03 14:00:00,1.026764,1.028770,57.926983,-0.002610,-0.003188,1.040713,1.016216,0.430550,0.002537
2025-02-03 15:00:00,1.024791,1.028392,50.338145,-0.002618,-0.003074,1.038145,1.017063,0.366599,0.002486
2025-02-03 16:00:00,1.023952,1.027969,53.006313,-0.002661,-0.002991,1.036626,1.017278,0.344966,0.002219
2025-02-03 17:00:00,1.024141,1.027604,51.450513,-0.002650,-0.002923,1.034608,1.017948,0.371702,0.002671


In [9]:
strategy = TradeStrategy(tk.predict_df, 2)
strategy.run(dataset.df)
strategy.df

,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT
2025-02-03 09:00:00,1.024310,1.028349,BUY,1.020857,1.040836
2025-02-03 10:00:00,1.028349,1.028318,SELL,1.034274,1.016404
2025-02-03 11:00:00,1.028318,1.028293,SELL,1.035655,1.016022
2025-02-03 12:00:00,1.028293,1.028214,SELL,1.035517,1.016044
2025-02-03 13:00:00,1.028214,1.026950,SELL,1.033970,1.015250
2025-02-03 14:00:00,1.026950,1.026764,SELL,1.031330,1.017630
2025-02-03 15:00:00,1.026764,1.024791,SELL,1.029267,1.015840
2025-02-03 16:00:00,1.024791,1.023952,SELL,1.027947,1.015963
2025-02-03 17:00:00,1.023952,1.024141,BUY,1.019333,1.033755


# Backtesting du model

In [10]:
strategy.backtesting(base_df[['HIGH', 'LOW', 'CLOSE']], 1000, 0.02)

Final Capital : 1000.00
Win trades : 0, Loss trades : 0
Success rate : 0.00%


,HIGH,LOW,CLOSE,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT
2025-02-03 09:00:00,1.02491,1.02219,1.02246,1.024310,1.028349,BUY,1.020857,1.040836
2025-02-03 10:00:00,1.02511,1.02129,1.02438,1.028349,1.028318,SELL,1.034274,1.016404
2025-02-03 11:00:00,1.02541,1.02256,1.02297,1.028318,1.028293,SELL,1.035655,1.016022
2025-02-03 12:00:00,1.02442,1.02296,1.02435,1.028293,1.028214,SELL,1.035517,1.016044
2025-02-03 13:00:00,1.02556,1.02363,1.02522,1.028214,1.026950,SELL,1.033970,1.015250
2025-02-03 14:00:00,1.02778,1.02514,1.02576,1.026950,1.026764,SELL,1.031330,1.017630
2025-02-03 15:00:00,1.02871,1.02576,1.02638,1.026764,1.024791,SELL,1.029267,1.015840
2025-02-03 16:00:00,1.02659,1.02472,1.02491,1.024791,1.023952,SELL,1.027947,1.015963
2025-02-03 17:00:00,1.03351,1.02416,1.03160,1.023952,1.024141,BUY,1.019333,1.033755


In [11]:
backtest_data = base_df[['CLOSE']].merge(strategy.df, left_index=True, right_index=True).merge(dataset.df[['ATR']], left_index=True, right_index=True)
backtest_data

,CLOSE,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT,ATR
2025-02-03 09:00:00,1.02246,1.024310,1.028349,BUY,1.020857,1.040836,0.003469
2025-02-03 10:00:00,1.02438,1.028349,1.028318,SELL,1.034274,1.016404,0.003309
2025-02-03 11:00:00,1.02297,1.028318,1.028293,SELL,1.035655,1.016022,0.003409
2025-02-03 12:00:00,1.02435,1.028293,1.028214,SELL,1.035517,1.016044,0.003381
2025-02-03 13:00:00,1.02522,1.028214,1.026950,SELL,1.033970,1.015250,0.003250
2025-02-03 14:00:00,1.02576,1.026950,1.026764,SELL,1.031330,1.017630,0.002537
2025-02-03 15:00:00,1.02638,1.026764,1.024791,SELL,1.029267,1.015840,0.002486
2025-02-03 16:00:00,1.02491,1.024791,1.023952,SELL,1.027947,1.015963,0.002219
2025-02-03 17:00:00,1.03160,1.023952,1.024141,BUY,1.019333,1.033755,0.002671


In [17]:
1.024310 + (0.003469 * 2)

1.0312480000000002

In [12]:
backtester = Backtester(backtest_data)
backtester.simulate_trades()
performance_metrics = backtester.calculate_performance_metrics()

print("Métriques de performance :")
for key, value in performance_metrics.items():
    print(f"{key}: {value}")

Métriques de performance :
total_trades: 9
winning_trades: 7
losing_trades: 2
win_rate: 0.78
net_profit: 0.02
average_profit: 0.0
sharpe_ratio: 0.88


In [13]:
# Test de différents multiplicateurs
multipliers = [(1.0, 1.5), (1.5, 2.0), (2.0, 2.5)]

best_metrics = None
best_multipliers = None

for stop_loss_multiplier, take_profit_multiplier in multipliers:
    # Appliquez les multiplicateurs et recalculez les stop-loss et take-profit
    backtest_data['STOP_LOSS'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] + (stop_loss_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] - (stop_loss_multiplier * row['ATR']),
        axis=1
    )
    backtest_data['TAKE_PROFIT'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] - (take_profit_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] + (take_profit_multiplier * row['ATR']),
        axis=1
    )

    # Simule les trades et calcule les métriques
    backtester = Backtester(backtest_data)
    backtester.simulate_trades()
    metrics = backtester.calculate_performance_metrics()

    # Compare les métriques pour trouver les meilleurs paramètres
    if best_metrics is None or metrics['net_profit'] > best_metrics['net_profit']:
        best_metrics = metrics
        best_multipliers = (stop_loss_multiplier, take_profit_multiplier)

print("Meilleurs multiplicateurs :", best_multipliers)
print("Métriques associées :", best_metrics)

Meilleurs multiplicateurs : (1.0, 1.5)
Métriques associées : {'total_trades': 9, 'winning_trades': 8, 'losing_trades': 1, 'win_rate': 0.89, 'net_profit': 0.02, 'average_profit': 0.0, 'sharpe_ratio': 1.26}
